# Odporúčanie

In [29]:
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# dependencies
import pandas as pd
import wandb

from _helpers import constants
from _helpers.verify_submission.verify_subm import main as verify_subm
from _helpers.score_submission.score_subm import main as score_subm

from drop import main as drop
from subset import main as subset

from models.model_random import ModelRandom
from models.model_nochange import ModelNoChange
from models.model_popular import ModelPopular
from models.model_log_reg import ModelLogisticRegression
from models.model_cheapest import ModelCheapest

Iitialize wandb - logging tool

In [31]:
wandb.login()

True

Load the correct data

In [32]:
train_file = constants.DROPPED_TRAIN
test_file = constants.DROPPED_TEST
ground_truth_file = constants.DROPPED_GROUND_TRUTH

if (constants.SUBSET is not None):
    train_file = constants.DROPPED_SUBSET(constants.SUBSET, 'train')
    test_file = constants.DROPPED_SUBSET(constants.SUBSET, 'test')
    ground_truth_file = constants.DROPPED_SUBSET(constants.SUBSET, 'test')

if not (train_file.exists() and test_file.exists() and ground_truth_file.exists()):
    if (constants.SUBSET is not None):
        subset()
    else:
        drop()

df_train = pd.read_parquet(train_file)

In [33]:
models = {
    'random': ModelRandom(),
    'nochange': ModelNoChange(),
    'popular': ModelPopular(),
    'log-reg': ModelLogisticRegression(),
    'cheapest': ModelCheapest(),
}

Choose the correct model and setup parameters of the model

In [60]:
# Tinker with the parameters
run = 9
notes = ''
params = {
    'model': 'log-reg',
    'subset': constants.SUBSET,
    'features': [
        "impressed_item_position",
        # "impressed_item_rating",
        # "user_impressed_item_interaction_count",
        "price",
        "price_above_impression_mean",
        "is_last_interacted",
    ]
}

wandb_run = wandb.init(entity='mcfreddie777', project="dp-recsys", name=f'model_{params["model"]}_run_{run}',
                       notes=notes)
wandb_run.config.update(params)

model = models[params['model']]
model.update(params)
model.fit(df_train)

Running __select_only_reference_action_columns...
Running __add_previous_item_column...
Running __explode_impressions_prices_columns...
Running __add_user_interacted_item_interaction_count_column...
Running __narrow_to_clickouts...
Running cast...
Running __add_mean_price_column...
Running __add_impressed_item_position_column...
Running __add_last_interacted_column...
Running __encode_cat_columns...
Running __add_rating_column...
Running __collect_features...
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.50109D+03    |proj g|=  3.46890D+05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gr

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Predikujeme odporúčania

In [61]:
df_test = pd.read_parquet(test_file)
df_recommendations = model.predict(df_test)

Running __select_only_reference_action_columns...
Running __add_previous_item_column...
Running __explode_impressions_prices_columns...
Running __add_user_interacted_item_interaction_count_column...
Running __narrow_to_clickouts...
Running cast...
Running __add_mean_price_column...
Running __add_impressed_item_position_column...
Running __add_last_interacted_column...
Running __encode_cat_columns...
Running __add_rating_column...
Running __collect_features...


Verifikujeme predikcie

In [62]:
verify_subm(df_subm=df_recommendations, df_test=df_test)

Checking for required columns in the submission file...
> check passed
Checking for duplicate sessions in the submission file...
> check passed
Checking that all the required sessions are present in submission...
> check passed
All checks passed


In [63]:
df_recommendations.to_csv(constants.OUTPUT_DIR / f'submission_popular_{params["model"]}_{run}.csv', index=False)

Vypočítame si MRR na dátach

In [64]:
df_gt = pd.read_csv(constants.GROUND_TRUTH)
df_test = pd.read_parquet(test_file)
mrr, map3 = score_subm(df_subm=df_recommendations, df_gt=df_gt)
wandb_run.log({"mrr": mrr, "map3": map3})

Mean reciprocal rank:      0.287
Mean average precision @3: 0.1033


In [65]:
wandb_run.finish()


map3,▁
mrr,▁
map3,0.1033
mrr,0.287
